In [5]:
import torch
import numpy as np
from utils import optm
from utils import optm_method as optmm
from utils import lossfunc as lf
from utils import functions as f

from importlib import reload
reload(optm)
reload(lf)

<module 'utils.lossfunc' from '/Users/keisukemurota/Documents/todo/worms/python/nsp/utils/lossfunc.py'>

### If we know the ground state

In [16]:
X = np.load("../array/MG_union_bond.npy")
X = (X + X.T)/2
Y = lf.positive_map_np(X) / 1.5

L = 2
H = np.zeros((2**(3*L), 2**(3*L)), dtype=np.float64)
for i in range(L):
    H += f.l2nl(X, L, [i, (i+1)%L], sps = 8)    
    
H_abs = np.zeros((2**(3*L), 2**(3*L)), dtype=np.float64)
for i in range(L):
    H_abs += f.l2nl(Y, L, [i, (i+1)%L], sps = 8) 
    
print("rate of ground state energy = ", np.linalg.eigvalsh(H_abs)[-1] / np.linalg.eigvalsh(H)[-1])
np.save("../array/MG_union_abs1.npy",Y)
np.save("../array/MG_union_bond1.npy", X)
np.save("../array/exact_MG.npy", np.linalg.eigvalsh(H))

rate of ground state energy =  0.9996004939667932


### If I know the ground state 2

In [26]:
X = np.load("../array/MG_union_rns_bond.npy")
X = (X + X.T)/2
Y = lf.positive_map_np(X) / 1.3910870555241834
L = 2
H = np.zeros((2**(3*L), 2**(3*L)), dtype=np.float64)
for i in range(L):
    H += f.l2nl(X, L, [i, (i+1)%L], sps = 8)    
    
H_abs = np.zeros((2**(3*L), 2**(3*L)), dtype=np.float64)
for i in range(L):
    H_abs += f.l2nl(Y, L, [i, (i+1)%L], sps = 8) 
    
print("rate of ground state energy = ", np.linalg.eigvalsh(H_abs)[-1] / np.linalg.eigvalsh(H)[-1])
np.save("../array/MG_union_abs2.npy",Y)
np.save("../array/MG_union_bond2.npy", X)

rate of ground state energy =  0.9999999999998589


In [28]:
Y[np.abs(Y) < 1E-8] = 0

array([2.98918909e-11, 3.14707371e-09, 5.36561134e-09, 2.90393727e-09,
       6.00966979e-09, 1.09080330e-09, 5.44210290e-09, 6.33347839e-09,
       6.02231238e-09, 2.48141027e-09, 3.54721045e-09, 2.28781947e-09,
       8.72504716e-09, 4.84375093e-10, 7.68551875e-09, 3.89177907e-09,
       1.91356861e-09, 1.39181926e-10, 8.47197533e-09, 7.81293483e-09,
       7.46120997e-09, 4.31252283e-09, 4.14587736e-09, 4.29513180e-09,
       4.06344265e-09, 4.79502975e-09, 1.28107267e-09, 2.39786564e-09,
       4.06705685e-09, 8.68373812e-09, 8.88944799e-11, 1.91356861e-09,
       9.46169760e-09, 7.84003421e-09, 2.48141027e-09, 2.28781947e-09,
       7.46120997e-09, 4.06705685e-09, 1.39181926e-10, 4.31252283e-09,
       4.06344265e-09, 7.77163187e-11, 8.72504716e-09, 8.68373812e-09,
       8.47197533e-09, 4.14587736e-09, 2.98918909e-11, 8.88944799e-11,
       4.29513180e-09, 4.79502975e-09, 6.00966979e-09, 3.54721045e-09,
       5.36561134e-09, 5.44210290e-09, 7.81293483e-09, 3.14707371e-09,
      

In [3]:
solver = optm.reweight_solver(X)
optimizer = torch.optim.Adam(solver.parameters(), lr = 0.0012, weight_decay=0)

In [253]:
E_t = solver.target
for t in range(10**3):
    Y = solver.matrix
    optimizer.zero_grad()
    loss, diff = lf.reweight_loss(Y, E_t, solver.X, lam = 0, prod = 0)
    loss.backward()
    optimizer.step()
    if t % 100 == 0:
        print("iteration : {:4d}   loss : {:.5f}".format(t,loss.item()), end="\t ")
        print("delta : {:.5f}".format(diff))
    

iteration :    0   loss : 2.31218	 delta : 2.31218
iteration :  100   loss : 1.78413	 delta : 1.78413
iteration :  200   loss : 1.38933	 delta : 1.38933
iteration :  300   loss : 1.02431	 delta : 1.02431
iteration :  400   loss : 0.72698	 delta : 0.72698
iteration :  500   loss : 0.45968	 delta : 0.45968
iteration :  600   loss : 0.21881	 delta : 0.21881
iteration :  700   loss : 0.00000	 delta : 0.00000
iteration :  800   loss : 0.00000	 delta : 0.00000
iteration :  900   loss : 0.00000	 delta : 0.00000


In [244]:
solver._params

Parameter containing:
tensor([ 1.1212,  0.7044,  0.0297,  0.0191,  0.2736, -0.0547, -0.1595, -0.2914,
        -0.1737, -0.1559, -0.1997, -0.2699, -0.2280, -0.2707, -0.2982, -0.0546,
         0.3705, -0.2046, -0.2305, -0.2793, -0.2335, -0.2792, -0.2745, -0.1635,
        -0.2342, -0.2837, -0.2724, -0.1573, -0.2707, -0.1185,  0.1527,  0.4593,
         0.6423,  0.1641, -0.2249, -0.3005, -0.1734, -0.2685, -0.2876, -0.2247,
        -0.2315, -0.2446, -0.2366, -0.2361, -0.2878, -0.1794, -0.2198,  0.1816,
        -0.0336, -0.1927, -0.2061, -0.2285, -0.2653, -0.2287, -0.1636,  0.0950,
        -0.3033, -0.1513, -0.2028,  0.1128,  0.0142,  0.2725,  0.4936,  0.2548],
       dtype=torch.float64, requires_grad=True)

In [255]:
Y = np.array(Y.data)
Y[np.abs(Y) < 1E-8] = 0
np.save("../array/MG_reweight.npy",Y)
X[np.abs(X) < 1E-8] = 0
np.save("../array/MG_reweight_ori.npy",X)


In [321]:
X = np.load("../array/MG_union_rns_bond.npy")
X = (X + X.T)/2
solver = optm.reweight_solver2(X)

In [322]:
optimizer = torch.optim.Adam(solver.parameters(), lr = 0.0012, weight_decay=0)
# optimizer = optmm.scheme1(solver.parameters(), gamma = 0.001)

In [323]:
E_t = solver.target
for t in range(3000):
    Y = solver.matrix
    optimizer.zero_grad()
    loss, diff = lf.reweight_loss(Y, E_t, solver.X, lam = 0.01, prod = 1, type_ = 2)
    loss.backward()
    optimizer.step()
    if t % 100 == 0:
        print("iteration : {:4d}   loss : {:.5f}".format(t,loss.item()), end="\t ")
        print("delta : {:.5f}".format(diff))
    

iteration :    0   loss : 41.98904	 delta : 0.87890
iteration :  100   loss : 41.18868	 delta : 0.78890
iteration :  200   loss : 41.03931	 delta : 0.71706
iteration :  300   loss : 40.94940	 delta : 0.65600
iteration :  400   loss : 40.89066	 delta : 0.60688
iteration :  500   loss : 40.85002	 delta : 0.56934
iteration :  600   loss : 40.82240	 delta : 0.54300
iteration :  700   loss : 40.80581	 delta : 0.52848
iteration :  800   loss : 40.79368	 delta : 0.51956
iteration :  900   loss : 40.78442	 delta : 0.51410
iteration : 1000   loss : 40.77786	 delta : 0.51196
iteration : 1100   loss : 40.77237	 delta : 0.51073
iteration : 1200   loss : 40.76813	 delta : 0.51045
iteration : 1300   loss : 40.76522	 delta : 0.51166
iteration : 1400   loss : 40.76283	 delta : 0.51284
iteration : 1500   loss : 40.76078	 delta : 0.51381
iteration : 1600   loss : 40.75903	 delta : 0.51472
iteration : 1700   loss : 40.75754	 delta : 0.51561
iteration : 1800   loss : 40.75626	 delta : 0.51645


KeyboardInterrupt: 

In [310]:
Y = np.array(Y.data)
Y[np.abs(Y) < 1E-8] = 0
np.save("../array/MG_reweight2.npy",Y)
X[np.abs(X) < 1E-8] = 0
np.save("../array/MG_reweight2_ori.npy",X)

In [271]:
X = np.load("../array/MG_union_bond.npy")
X = (X + X.T)/2
solver = optm.reweight_solver3(X)

In [211]:
optimizer = torch.optim.Adam(solver.parameters(), lr = 0.0012, weight_decay=0)
E_t = solver.target
for t in range(10**4):
    Y = solver.matrix
    optimizer.zero_grad()
    loss, diff = lf.reweight_loss(Y, E_t, solver.X, lam = 0.001, prod = 0, type_ = 0)
    loss.backward()
    optimizer.step()
    if t % 100 == 0:
        print("iteration : {:4d}   loss : {:.5f}".format(t,loss.item()), end="\t ")
        print("delta : {:.5f}".format(diff))
    

iteration :    0   loss : 6.31825	 delta : 6.20764
iteration :  100   loss : 5.40865	 delta : 5.28614
iteration :  200   loss : 4.81843	 delta : 4.68148
iteration :  300   loss : 4.29884	 delta : 4.14888
iteration :  400   loss : 3.81432	 delta : 3.64934
iteration :  500   loss : 3.36409	 delta : 3.18157
iteration :  600   loss : 2.94006	 delta : 2.73787
iteration :  700   loss : 2.53932	 delta : 2.31593
iteration :  800   loss : 2.15896	 delta : 1.91114
iteration :  900   loss : 1.80597	 delta : 1.52894
iteration : 1000   loss : 1.50649	 delta : 1.20259
iteration : 1100   loss : 1.26566	 delta : 0.93656
iteration : 1200   loss : 1.06067	 delta : 0.70728
iteration : 1300   loss : 0.89764	 delta : 0.52100
iteration : 1400   loss : 0.75498	 delta : 0.35382
iteration : 1500   loss : 0.63744	 delta : 0.21004
iteration : 1600   loss : 0.53684	 delta : 0.07707
iteration : 1700   loss : 0.47443	 delta : 0.00111
iteration : 1800   loss : 0.46600	 delta : 0.00000
iteration : 1900   loss : 0.460

KeyboardInterrupt: 

In [212]:
Y = np.array(Y.data)
Y[Y < 1E-8] = 0
np.save("../array/MG_reweight3.npy",Y)
X[np.abs(X) < 1E-8] = 0
np.save("../array/MG_reweight3_ori.npy",X)

In [219]:
EY = np.linalg.eigvalsh(Y.data)
EX = np.linalg.eigvalsh(X.data)

np.exp(EX).sum() / np.exp(EY).sum()

0.8888140795434099

In [222]:
np.diag(Y)

array([1.11551769, 0.56204556, 0.32855347, 0.30225631, 0.16581119,
       0.15189786, 0.1380661 , 0.16770426, 0.15340152, 0.21122883,
       0.21250688, 0.2177844 , 0.20050037, 0.20182089, 0.14387668,
       0.30028006, 0.36214958, 0.14886978, 0.20341057, 0.20188606,
       0.21472373, 0.20793637, 0.20283472, 0.15024733, 0.1396788 ,
       0.19212188, 0.20282829, 0.20946463, 0.14388069, 0.15120215,
       0.37091581, 0.43874344, 0.57131021, 0.37048926, 0.1486761 ,
       0.1438847 , 0.21096139, 0.20286833, 0.19217128, 0.13627412,
       0.15387374, 0.20288089, 0.20800944, 0.21106183, 0.20193677,
       0.20177367, 0.14728428, 0.3250694 , 0.3041348 , 0.14389172,
       0.2019065 , 0.19958041, 0.21781582, 0.21141558, 0.20933663,
       0.37702547, 0.16772874, 0.1373965 , 0.15129395, 0.16440857,
       0.30069904, 0.3624945 , 0.57816265, 1.11281675])

In [223]:
np.diag(X)

array([-1.125, -0.875, -0.375, -0.375,  0.125,  0.125,  0.125, -0.125,
        0.125,  0.375,  0.375,  0.375,  0.375,  0.375, -0.125, -0.375,
       -0.375, -0.125,  0.375,  0.375,  0.375,  0.375,  0.375,  0.125,
        0.125,  0.375,  0.375,  0.375, -0.125, -0.125, -0.625, -0.875,
       -0.875, -0.625, -0.125, -0.125,  0.375,  0.375,  0.375,  0.125,
        0.125,  0.375,  0.375,  0.375,  0.375,  0.375, -0.125, -0.375,
       -0.375, -0.125,  0.375,  0.375,  0.375,  0.375,  0.375,  0.125,
       -0.125,  0.125,  0.125,  0.125, -0.375, -0.375, -0.875, -1.125])